In [ ]:
!pip install -U langchain-community


In [ ]:
! pip install langchain

In [ ]:
!pip install -q --upgrade transformers accelerate
! pip install sentencepiece

In [ ]:
!pip uninstall bitsandbytes -y


In [ ]:
! pip install bitsandbytes


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "NousResearch/Hermes-2-Pro-Mistral-7B"

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=False,
    trust_remote_code=True,
    revision="main"
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
    revision="main"
)


2025-10-12 16:32:04.308790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760286724.331750     103 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760286724.338830     103 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
from transformers  import AutoTokenizer , AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

pipe=pipeline("text-generation",
            model=base_model,
            tokenizer=tokenizer,
            temperature=0.2
)

llms =HuggingFacePipeline(pipeline=pipe)


Device set to use cuda:0
/tmp/ipykernel_103/897052204.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llms =HuggingFacePipeline(pipeline=pipe)


# Fewshot prompt

## without langchin

In [ ]:
demo_message = """
Country Name: Egypt | Capital Name: Cairo
Country Name: Saudi Arabia | Capital Name: Riyadh
Country Name: France | Capital Name: Paris
Country Name: China | Capital Name:
""".strip()

print(llms(demo_message))

/tmp/ipykernel_103/1655303284.py:8: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llms(demo_message))


Country Name: Egypt | Capital Name: Cairo
Country Name: Saudi Arabia | Capital Name: Riyadh
Country Name: France | Capital Name: Paris
Country Name: China | Capital Name: Beijing
Country Name: Japan | Capital Name: Tokyo
Country Name: Germany | Capital Name: Berlin
Country Name: United Kingdom | Capital Name: London
Country Name: India | Capital Name: New Delhi
Country Name: Australia | Capital Name: Canberra
Country Name: Canada | Capital Name: Ottawa
Country Name: United States | Capital Name: Washington, D.C.
Country Name: Brazil | Capital Name: Brasília
Country Name: Italy | Capital Name: Rome
Country Name: Spain | Capital Name: Madrid
Country Name: Mexico | Capital Name: Mexico City
Country Name: South Korea | Capital Name: Seoul
Country Name: Turkey | Capital Name: Ankara
Country Name: Russia | Capital Name: Moscow
Country Name: Ukraine | Capital Name: Kyiv
Country Name: Argentina | Capital Name: Buenos Aires
Country Name: Colombia | Capital Name: Bogotá
Country Name: Peru | Capi

## using langchian

In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# Example pattern
demo_template = PromptTemplate(
    template="Country Name: {country} | Capital Name: {capital}",
    input_variables=["country", "capital"]
)

# Few-shot examples (just to teach the pattern)
examples = [
    {"country": "Egypt", "capital": "Cairo"},
    {"country": "Saudi Arabia", "capital": "Riyadh"},
    {"country": "France", "capital": "Paris"},
]

# Few-shot prompt
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=demo_template,
    suffix="Now, write only for this one country:\nCountry Name: {country} | Capital Name: (Write only the capital name for this country only, not others)",
    input_variables=["country"]
)

In [ ]:

formatted = prompt.format(country="Germany")
print(formatted)

Country Name: Egypt | Capital Name: Cairo

Country Name: Saudi Arabia | Capital Name: Riyadh

Country Name: France | Capital Name: Paris

Now, write only for this one country:
Country Name: Germany | Capital Name: (Write only the capital name for this country only, not others)


In [ ]:
print( llms( prompt.format(country='Germany') ) )

Country Name: Egypt | Capital Name: Cairo

Country Name: Saudi Arabia | Capital Name: Riyadh

Country Name: France | Capital Name: Paris

Now, write only for this one country:
Country Name: Germany | Capital Name: (Write only the capital name for this country only, not others) Berlin

Country Name: Japan | Capital Name: (Write only the capital name for this country only, not others) Tokyo

Country Name: Australia | Capital Name: (Write only the capital name for this country only, not others) Canberra

Country Name: United Kingdom | Capital Name: (Write only the capital name for this country only, not others) London

Country Name: Italy | Capital Name: (Write only the capital name for this country only, not others) Rome

Country Name: Spain | Capital Name: (Write only the capital name for this country only, not others) Madrid

Country Name: Brazil | Capital Name: (Write only the capital name for this country only, not others) Brasília

Country Name: Argentina | Capital Name: (Write only

# Reasoing problem

In [ ]:
riddle_1 = """
I have five oranges, and the sum of what my sister and brother have is three times what I have plus thirty-five
""".strip()

print( llms( riddle_1 ) )

I have five oranges, and the sum of what my sister and brother have is three times what I have plus thirty-five. How many oranges do my sister and brother have?

To solve this problem, I need to find a way to express the relationship between what I have and what they have using algebra. Let me use x to represent the number of oranges I have. Then, the number of oranges my sister has is y, and the number of oranges my brother has is z. The problem says that the sum of what my sister and brother have is three times what I have plus thirty-five. This can be written as an equation: y + z = 3x + 35. But I also know that I have five oranges, so x = 5. I can substitute this into the equation and get: y + z = 3(5) + 35. Simplifying this, I get: y + z = 40. This means that my sister and brother together have forty oranges. But the problem asks for the number of oranges each of them has individually. To find that, I need to use another piece of information from the problem: the sum of what my si

# using fewshot prompt

In [ ]:
examples = [
    {
        "question": "When I was seven, my sister was twice my age. Now I am seventy years old, how old can my sister be?",
        "answer": "\n".join([
            "We will followup some questions to get the answer.",
            "Follow up: How old was your sister when you were seven?",
            "Intermediate answer: Twice, which mean 14 years.",
            "Follow up: What is the difference between your age and your sister's age?",
            "Intermediate answer: 14 years - 7 years = 7 years.",
            "Follow up: When you were seventy years old, how old would your sister be?",
            "Intermediate answer: my age (70) +  The difference between me and my sister's age (7) = 77 years.",
            "Final Answer: 77 years."
        ])
    },
    {
        "question": "I have five oranges, and the sum of what my sister and brother have is three times what I have plus thirty-five?",
        "answer": "\n".join([
            "We will followup some questions to get the answer.",
            "Follow up: How many oranges do you have?",
            "Intermediate answer: 5 oranges.",
            "Follow up: How many oranges does your sister and brother have?",
            "Intermediate answer: three times: = 3 * 5 = 15. Also, we need to add thirty-five to them: 15 + 35 = 50 ",
            "Final Answer: 50 oranges."
        ])
    }
]

In [ ]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

message = prompt.format(input="Total with what my family is 50 oranges. If we subtract one-fifth of the number from them, and add ten more oranges, how many oranges will there be in the end?")
print( llms(message) )

Question: When I was seven, my sister was twice my age. Now I am seventy years old, how old can my sister be?
We will followup some questions to get the answer.
Follow up: How old was your sister when you were seven?
Intermediate answer: Twice, which mean 14 years.
Follow up: What is the difference between your age and your sister's age?
Intermediate answer: 14 years - 7 years = 7 years.
Follow up: When you were seventy years old, how old would your sister be?
Intermediate answer: my age (70) +  The difference between me and my sister's age (7) = 77 years.
Final Answer: 77 years.

Question: I have five oranges, and the sum of what my sister and brother have is three times what I have plus thirty-five?
We will followup some questions to get the answer.
Follow up: How many oranges do you have?
Intermediate answer: 5 oranges.
Follow up: How many oranges does your sister and brother have?
Intermediate answer: three times: = 3 * 5 = 15. Also, we need to add thirty-five to them: 15 + 35 = 50

## another problem

In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

demo_prompt = PromptTemplate(template="What're the ingredients to cook a pizza?", input_variables=[])

demo_prompt_message = demo_prompt.format()

llms( demo_prompt_message )

"What're the ingredients to cook a pizza?\n\nThe ingredients to cook a pizza are:\n\n1. Pizza dough\n2. Pizza sauce\n3. Cheese (usually mozzarella)\n4. Toppings of your choice (pepperoni, mushrooms, onions, bell peppers, olives, etc.)\n5. Olive oil\n6. Flour or cornmeal (for dusting the pizza peel)\n\nYou can also add herbs and spices like oregano, basil, garlic, etc. to enhance the flavor of the pizza.\n\nHow do you cook a pizza in an oven?\n\nHere are the steps to cook a pizza in an oven:\n\n1. Preheat the oven to 450-500 degrees Fahrenheit.\n2. Roll out the pizza dough on a floured surface and transfer it to a pizza peel dusted with flour or cornmeal.\n3. Brush the edges of the dough with olive oil and add the pizza sauce, spreading it evenly with the back of a spoon.\n4. Add the cheese and toppings of your choice.\n5. Slide the pizza onto a prehe"

## using fewshot prompt    

In [ ]:
examples = [
    {"recipe": "olives pizza", "ingredients": "\n".join([
                                    "* 1/2 pizza dough recipe",
                                    "* 1 tablespoon olive oil",
                                    "* 8 ounces cream cheese",
                                    "* 1/2 cup mozzarella cheese shredded",
                                    "* 1 tablespoon fresh parsley",
                                    "* 1/3 cup sliced green olives",
                                    "* 1/3 cup sliced black olives"
                                ])},
    {"recipe":"falafel", "ingredients": "\n".join(
                                [
                                    "* 1 cup dried chickpeas, soaked overnight",
                                    "* 1/2 cup onion",
                                    "* 1 cup parsley",
                                    "* 1 cup cilantro",
                                    "* 1 tablespoon fresh parsley",
                                    "* 1/3 cup sliced green olives",
                                    "* 1/3 cup sliced black olives",
                                    "* 3 garlic cloves",
                                    "* 1 tsp salt"
                                ]
                                )}
]

In [ ]:
example_prompt = PromptTemplate(
    template="Recipe Name: {recipe}\nIngredients: {ingredients}",
    input_variables=['recipe', 'ingredients']
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Recipe Name: {recipe}\n",
    input_variables=['recipe']
)


In [ ]:
user_recipe = 'mashed potato'

message = prompt.format(recipe=user_recipe)

print( llms( message ) )

Recipe Name: olives pizza
Ingredients: * 1/2 pizza dough recipe
* 1 tablespoon olive oil
* 8 ounces cream cheese
* 1/2 cup mozzarella cheese shredded
* 1 tablespoon fresh parsley
* 1/3 cup sliced green olives
* 1/3 cup sliced black olives

Recipe Name: falafel
Ingredients: * 1 cup dried chickpeas, soaked overnight
* 1/2 cup onion
* 1 cup parsley
* 1 cup cilantro
* 1 tablespoon fresh parsley
* 1/3 cup sliced green olives
* 1/3 cup sliced black olives
* 3 garlic cloves
* 1 tsp salt

Recipe Name: mashed potato
Ingredients: * 4 large russet potatoes
* 1/2 cup milk
* 1/4 cup butter
* 1 teaspoon salt
* 1/4 teaspoon black pepper
* 1/4 cup sliced green olives
* 1/4 cup sliced black olives

Recipe Name: stuffed bell peppers
Ingredients: * 4 large bell peppers
* 1 pound ground beef
* 1 onion, chopped
* 2 cloves garlic, minced
* 1 (14.5 ounce) can diced tomatoes
* 1/2 cup rice
* 1/2 cup water
* 1 teaspoon salt
* 1/4 teaspoon black pepper
* 1/4 cup sliced green olives
* 1/4 cup sliced black olives

# Important NOTE
- fewshot take many token , if find other way is good

In [ ]:
print (message)
# is very token , just is 2 EX

Recipe Name: olives pizza
Ingredients: * 1/2 pizza dough recipe
* 1 tablespoon olive oil
* 8 ounces cream cheese
* 1/2 cup mozzarella cheese shredded
* 1 tablespoon fresh parsley
* 1/3 cup sliced green olives
* 1/3 cup sliced black olives

Recipe Name: falafel
Ingredients: * 1 cup dried chickpeas, soaked overnight
* 1/2 cup onion
* 1 cup parsley
* 1 cup cilantro
* 1 tablespoon fresh parsley
* 1/3 cup sliced green olives
* 1/3 cup sliced black olives
* 3 garlic cloves
* 1 tsp salt

Recipe Name: mashed potato

